In [11]:
from joblib import dump, load
import pandas as pd
import re
from tabulate import tabulate

# =============================================
# 1. DADOS ORIGINAIS (FONTE)
# =============================================
df_original = pd.DataFrame({
    'id': [10003, 10002, 10001],
    'nome_fantasia': ['L03 - SALVADOR', 'L02 - SOBRAL', 'L01 - FORTALEZA'],
    'status': ['sim', 'sim', 'sim']
})

# =============================================
# 2. PROCESSAMENTO (NORMALIZAÇÃO)
# =============================================
df_processado = df_original.copy()

# Padronização de nomes de colunas (snake_case)
df_processado = df_processado.rename(columns={
    'nome_fantasia': 'unidade_nome',
    'status': 'unidade_status'
})

# Normalização de dados (1NF)
df_processado['unidade_nome'] = (
    df_processado['unidade_nome']
    .str.strip()
    .str.lower()
)

# Extração do número da loja (como função para evitar redundância)
def extrair_numero(unidade):
    match = re.search(r'l(\d+)', unidade)
    return match.group(1) if match else None

# Padronização de status (valores consistentess)
df_processado['unidade_status'] = (
    df_processado['unidade_status']
    .replace({'sim': 'ativo', 'não': 'inativo'})
    .astype('category')
)

# Tipos de dados otimizados
df_processado = df_processado.astype({
    'id': 'int32',
    'unidade_nome': 'string'
})

# =============================================
# 3. ESTRUTURA DE SAÍDA COM METADADOS TABULARES
# =============================================
metadados_tabulados = {
    'Processamento': [
        ['Data/Hora', pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')],
        ['Sistema', 'Normalização de Unidades v1.0']
    ],
    'Padronização': [
        ['Convenção Colunas', 'snake_case'],
        ['Status Permitidos', 'ativo, inativo'],
        ['Formato Texto', 'lowercase']
    ],
    'Normalização': [
        ['Forma Normal', '1NF'],
        ['Chave Primária', 'id'],
        ['Dependências', 'unidade_numero → unidade_nome']
    ],
    'Estatísticas': [
        ['Total Unidades', len(df_processado)],
        ['Unidades Ativas', sum(df_processado['unidade_status'] == 'ativo')],
        ['Tipos de Dados', str(df_processado.dtypes.to_dict())]
    ]
}

output_data = {
    'fonte': df_original,
    'normalizado': df_processado,
    'metadados': metadados_tabulados,
    'funcoes_uteis': {
        'extrair_numero_loja': extrair_numero
    }
}

# =============================================
# 4. PERSISTÊNCIA E VISUALIZAÇÃO
# =============================================
try:
    # Validação
    assert not df_processado.duplicated('id').any(), "IDs duplicados"
    assert df_processado['unidade_status'].isin(['ativo', 'inativo']).all(), "Status inválido"

    dump(output_data, 'padronizado_unidades.z')

    # Exibição formatada
    print("═"*60)
    print("METADADOS ORGANIZADOS".center(60))
    print("═"*60)

    for categoria, dados in metadados_tabulados.items():
        print(f"\n■ {categoria.upper()}")
        print(tabulate(dados, headers=['Descrição', 'Valor'], tablefmt='grid'))

    print("\n✓ Arquivo 'padronizado_unidades.z' gerado com sucesso!")

except Exception as e:
    print(f"Erro: {str(e)}")

════════════════════════════════════════════════════════════
                   METADADOS ORGANIZADOS                    
════════════════════════════════════════════════════════════

■ PROCESSAMENTO
+-------------+-------------------------------+
| Descrição   | Valor                         |
+=============+===============================+
| Data/Hora   | 2025-05-25 21:17:38           |
+-------------+-------------------------------+
| Sistema     | Normalização de Unidades v1.0 |
+-------------+-------------------------------+

■ PADRONIZAÇÃO
+-------------------+----------------+
| Descrição         | Valor          |
+===================+================+
| Convenção Colunas | snake_case     |
+-------------------+----------------+
| Status Permitidos | ativo, inativo |
+-------------------+----------------+
| Formato Texto     | lowercase      |
+-------------------+----------------+

■ NORMALIZAÇÃO
+----------------+-------------------------------+
| Descrição      | Valor      

In [13]:
from joblib import load
import pandas as pd

def display_unidades_data(file_path):

    try:
        data = load(file_path)
    except Exception as e:
        print(f"Erro ao carregar {file_path}: {str(e)}")
        return

    # Configurações de display
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.colheader_justify', 'center')

    def print_section(title, content):
        border = "═" * 60
        print(f"\n{border}")
        print(f"{title.center(60)}")
        print(f"{border}")
        if content is not None:
            if isinstance(content, pd.DataFrame):
                display(content)
            else:
                print(content)

    # Verifica estrutura do arquivo e exibe adequadamente
    if 'fonte' in data and 'normalizado' in data:  # Se for a nova estrutura
        print_section("DADOS ORIGINAIS (FONTE)", data['fonte'])
        print_section("DADOS NORMALIZADOS", data['normalizado'])
        print_section("METADADOS", data.get('metadados', 'Nenhum metadado disponível'))

    elif 'df_original' in data and 'df_processado' in data:  # Estrutura antiga
        print_section("DADOS ORIGINAIS", data['df_original'])
        print_section("DADOS PROCESSADOS", data['df_processado'])
        print_section("METADADOS", data.get('metadata', 'Nenhum metadado disponível'))

    else:  # Estrutura desconhecida
        print("\nESTRUTURA DO ARQUIVO:")
        print(f"Chaves encontradas: {list(data.keys())}")
        print("\nCONTEÚDO COMPLETO:")
        for key, value in data.items():
            print_section(key.upper(), value)

# Verifica se o arquivo existe antes de tentar carregar
import os
file_path = 'padronizado_unidades.z'
if os.path.exists(file_path):
    display_unidades_data(file_path)
else:
    print(f"Arquivo {file_path} não encontrado no diretório atual.")
    print(f"Diretório atual: {os.getcwd()}")


════════════════════════════════════════════════════════════
                  DADOS ORIGINAIS (FONTE)                   
════════════════════════════════════════════════════════════


,id,nome_fantasia,status
0,10003,L03 - SALVADOR,sim
1,10002,L02 - SOBRAL,sim
2,10001,L01 - FORTALEZA,sim



════════════════════════════════════════════════════════════
                     DADOS NORMALIZADOS                     
════════════════════════════════════════════════════════════


,id,unidade_nome,unidade_status
0,10003,l03 - salvador,ativo
1,10002,l02 - sobral,ativo
2,10001,l01 - fortaleza,ativo



════════════════════════════════════════════════════════════
                         METADADOS                          
════════════════════════════════════════════════════════════
{'Processamento': [['Data/Hora', '2025-05-25 21:17:38'], ['Sistema', 'Normalização de Unidades v1.0']], 'Padronização': [['Convenção Colunas', 'snake_case'], ['Status Permitidos', 'ativo, inativo'], ['Formato Texto', 'lowercase']], 'Normalização': [['Forma Normal', '1NF'], ['Chave Primária', 'id'], ['Dependências', 'unidade_numero → unidade_nome']], 'Estatísticas': [['Total Unidades', 3], ['Unidades Ativas', 3], ['Tipos de Dados', "{'id': dtype('int32'), 'unidade_nome': string[python], 'unidade_status': CategoricalDtype(categories=['ativo'], ordered=False, categories_dtype=object)}"]]}
